In [3]:
import torch
from transformer.transformer import Transformer

In [4]:
former = Transformer(
    d_model=64,
    nhead=2,
    num_encoder_layers=2,
    num_decoder_layers=2,
    d_ffn=256,
    dropout=0.1,
)

former

Transformer()

In [ ]:
src = torch.rand((32, 10, 512))
tgt = torch.rand((32, 20, 512))

causal_mask = former.generate_square_subsequent_mask()
out = former(src, tgt, tgt_mask=causal_mask)